In [1]:
cd /Users/keonkookkim/Documents

/Users/keonkookkim/Documents


In [2]:
!xattr -d com.apple.quarantine chromedriver # 맥에서 크롬 브라우저 허용하는 코드 

xattr: chromedriver: No such xattr: com.apple.quarantine


In [3]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys # enter 치는 명령어
import csv 
import pandas as pd 

In [74]:
# 브라우저 생성 
browser = webdriver.Chrome('/Users/keonkookkim/Documents/chromedriver')

# 웹 사이트 열기 
browser.get('https://www.yogiyo.co.kr/mobile/#/')

In [68]:
# 검색창 클릭
search = browser.find_element_by_xpath('//*[@id="search"]/div/form/input')
search.click()

In [6]:
# 검색어 입력
search.send_keys('서울특별시 서초구 반포동 20-45 반포자이플라자')
search.send_keys(Keys.ENTER)
time.sleep(15)

In [15]:
# 스크롤 전 높이
before_h = browser.execute_script("return window.scrollY") # 현재 스크롤 된 높이를 계산

# 무한 반복문 사용
while True:
    # 맨 아래로 스크롤을 내린다.
    browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
    
    # 스크롤 사이 페이지 로딩 시간 추가
    time.sleep(3)
    
    # 스크롤 후 높이 확인 
    after_h = browser.execute_script("return window.scrollY")
    
    # before_h = after_h # after의 높이를 before로 갱신
    
    if after_h > 10000: # 스크롤 높이보다 높으면 멈춤 
        break
    
    ########################### 스크롤 높이 설정 필요 ###########################

In [97]:
# # 가게 클릭 
# search = browser.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div[2]/div[1]/div')
# search.click()
# time.sleep(10)


In [98]:
# # 리뷰탭 클릭 
# search = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
# search.click()
# time.sleep(10)

In [39]:
# star_count = 0
# for j in range(1,6):
#     star = browser.find_element_by_xpath(f'//*[@id="review"]/li[16]/div[2]/div/span[1]/span[{j}]').get_attribute("class")
#     if star == 'full ng-scope':
#         star_count += 1
        
#     else :
#         star_count += 0
# print(star_count)

5


# 실행시 파일 초기화됨 주의!!!

In [6]:
# # 파일 생성 
# f = open(r"/Users/keonkookkim/Desktop/KKK/Selenium/review.csv", 'w', encoding="UTF-8", newline="") # 파일 입출력 기능 

# csvWirter = csv.writer(f) # wirter에 파일(f) 넘겨주기
# f.close()

In [168]:
# # 크롤링 하면서 더보기 누르는 버전 

# def review_crawling():
#     count = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]').text
    
#     time.sleep(2)
#     num = 0
#     for i in range(2, int(count)+1):
#         # 가게 이름
#         title = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
        
#         items = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]')
#         review = items.find_element_by_css_selector("p.ng-binding").text
#         ########################## 평점 추가 고민 해보기 ##########################
        
#         print(title,review)
#         csvWirter.writerow([title, review]) # 행 추가
#         num += 1
#         time.sleep(0.5)
#         if num % 10 == 0 :
#             browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
#             time.sleep(5)
#             # 더보기 클릭
#             more_click = browser.find_element_by_css_selector('li.list-group-item.btn-more')
#             more_click.click()
#             time.sleep(5)  

----
### 리뷰 크롤링
----

In [64]:
# 더보기를 미리 다 눌러놓고 크롤링하는 버전 
# 이게 휠씬 빠름

def reveiw_crawling_first_scroll():
    while True:
        try:          
            browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
            time.sleep(1)
            
            # 더보기 클릭
            more_click = browser.find_element_by_css_selector('li.list-group-item.btn-more')
            isExistNextPage = more_click.is_enabled() # 더보기 클릭 가능하다면
            if (isExistNextPage == True):
                more_click.click()
                time.sleep(2)  
            else :  # 없다면 끝 
                break
        except:
            break

    count = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]').text
    if int(count) >= 2: # 리뷰 개수가 2개 이하면 크롤링X
        for i in range(2, int(count)+1):
            # 가게 이름
            title = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
            
            items = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]')
            review = items.find_element_by_css_selector("p.ng-binding").text
            
            # 별 개수 세기
            star_count = 0
            for j in range(1,6):
                star = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[1]/span[{j}]').get_attribute("class")
                if star == 'full ng-scope':
                    star_count += 1
                    
                else :
                    star_count += 0
            # 맛 별점
            taste = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[3]').text
            # 양 별점 
            quantity = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[6]').text
            # 배달 별점
            delivery = quantity = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[9]').text
            
            print(title,'|',review,'|',star_count,'|',taste,'|',quantity,'|',delivery)
            csvWirter.writerow([title, review, star_count, taste, quantity, delivery]) # 행 추가

---
### 별점 낮은것 크롤링
---

In [65]:
def bad_reveiw_crawling():
    while True:
        try:          
            browser.find_element_by_css_selector("body").send_keys(Keys.END) # end키를 통해 스크롤을 맨 아래로 내림 
            time.sleep(1)
            
            # 더보기 클릭
            more_click = browser.find_element_by_css_selector('li.list-group-item.btn-more')
            isExistNextPage = more_click.is_enabled() # 더보기 클릭 가능하다면
            if (isExistNextPage == True):
                more_click.click()
                time.sleep(2)  
            else :  # 없다면 끝 
                break
        except:
            break

    count = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]').text
    if int(count) >= 2: # 리뷰 개수가 2개 이하면 크롤링X
        for i in range(2, int(count)+1):
            # 가게 이름
            title = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
            
            items = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]')
            review = items.find_element_by_css_selector("p.ng-binding").text
            
            # 별 개수 세기
            star_count = 0
            for j in range(1,6):
                star = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[1]/span[{j}]').get_attribute("class")
                if star == 'full ng-scope':
                    star_count += 1
                    
                else :
                    star_count += 0
            
            if star_count < 3 :
                
                 # 맛 별점
                taste = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[3]').text
                # 양 별점 
                quantity = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[6]').text
                # 배달 별점
                delivery = quantity = browser.find_element_by_xpath(f'//*[@id="review"]/li[{i}]/div[2]/div/span[2]/span[9]').text
                
                print(title,'|',review,'|',star_count,'|',taste,'|',quantity,'|',delivery)
                csvWirter.writerow([title, review, star_count, taste, quantity, delivery]) # 행 추가

In [7]:
# 파일에 데이터 덮어쓰기
f = open(r"/Users/keonkookkim/Desktop/KKK/Selenium/review.csv", 'a', encoding="UTF-8", newline="") # 파일 입출력 기능 

csvWirter = csv.writer(f) # wirter에 파일(f) 넘겨주기

---
# 지역 선택 후 전체 식당 크롤링
---

In [70]:

# 슈퍼레드위크 추천 크롤링
for i in range(1, 20):
    data = pd.read_csv('/Users/keonkookkim/Desktop/KKK/Selenium/review.csv')
    data.columns =['store','review','total_scope', 'taste_star','quantity_star','delivery_star']
    store_list = data['store'].unique()
    try:
        # 가게 클릭 
        search = browser.find_element_by_xpath(f'//*[@id="content"]/div/div[3]/div/div[2]/div[{i}]/div')
        search.click()
        time.sleep(5)
        store_name = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
        if store_name not in store_list:
        # 리뷰탭 클릭 
            search = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            search.click()
            time.sleep(5)
            
            #### 리뷰 크롤링
            # reveiw_crawling_first_scroll()
            bad_reveiw_crawling()
            
            # 뒤로가기
            browser.back()
            time.sleep(6)
        else :
            browser.back()
            time.sleep(6) 
    except:
        break

# 일반 등록 음식점 크롤링
for i in range(1,20):    
    data = pd.read_csv('/Users/keonkookkim/Desktop/KKK/Selenium/review.csv')
    data.columns =['store','review','total_scope', 'taste_star','quantity_star','delivery_star']
    store_list = data['store'].unique()
    try:
        # 가게 클릭
        search = browser.find_element_by_xpath(f'//*[@id="content"]/div/div[4]/div/div[2]/div[{i}]/div')
        search.click()
        time.sleep(5)        
        store_name = browser.find_element_by_css_selector('span.restaurant-name.ng-binding').text
        if store_name not in store_list:
        # 리뷰탭 클릭 
            search = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            search.click()
            time.sleep(5)
            
            #### 리뷰 크롤링
            # reveiw_crawling_first_scroll()
            bad_reveiw_crawling()
                
            # 뒤로가기
            browser.back()
            time.sleep(6)
        else :
            browser.back()
            time.sleep(6)
            
    except:
        break

오구쌀피자-복수점 | 피자가 다른 곳에비해 바로 식어 버리네요 딱딱해서 별루네요 그닦 맛이 있는지도 모르겠네요 | 2 | 1 | 2 | 2
오구쌀피자-복수점 | 배달을 쓰세요 다식은거 여럿사람 먹이실거에요? | 1 | 3 | 1 | 1
오구쌀피자-복수점 | 처음시켰는데 너무 실망이에요 베이컨듬뿍피자라해서 시켰는데 베이컨이 전혀 많지도 않고 크림소스로 시켰는데 맛도 없어요 스파게티는 그나마 나은데 맹맹해요 그리고 버섯 못먹어서 요청사항에 모든 메뉴에 버섯 전부 빼달라고 썼는데 피자, 스파게티 둘다 버섯이 잔뜩 있네요;; 배달도 예상시간보다 늦게온것도 모자라 문도 안두드리고 문자도 안남겨주고 그냥 가셨어요 모든면에서 실망이네요 | 2 | 2 | 2 | 2
오구쌀피자-복수점 | ㅇㅏ~~~ 진짜 맛없네ㅠㅠ | 1 | 1 | 3 | 3
오구쌀피자-복수점 | 그냥그냥 보기좋은떡 맛도좋다구하든데 보긴맛있어보이지만 보이느것만큼 맛은 별로고 치즈도 좀 부족한것같구.금액대비 뵬로입니다. | 2 | 2 | 2 | 2
피자알볼로-서대전점 | 배달이 넘 늦어요..약속한 시간은 오바하셨구 출발했다했는데 오지 않구. 맛은 있었지만 기다리느라 힘들었네요. | 1 | 4 | 1 | 1
피자알볼로-서대전점 | 할인쿠폰에 현혹된게 잘못이네요. 쿠폰주고 치즈를 이렇게 얇게. 도우바닥이 보이게 만들어줄줄이야. . . 배달팁 아까워서 할인쿠폰보고 신났다가 ㅠㅠ 이렇게하시면 결국은 가게로 영향이 가겠죠! 앞으로는 피자알볼로는 끈으려구요 | 2 | 2 | 5 | 5
피자알볼로-서대전점 | 배달 개늦고 피자 다 식어서옴 걍 최악임 | 1 | 1 | 1 | 1
피자알볼로-서대전점 | 집에서 꾀먼곳에서 배달온줄....스파게티는 굳어있고 피자는 미지근하고 위치를 찾아보니 가까운곳에 있더군요... | 2 | 2 | 2 | 2
피자알볼로-서대전점 | 구성 좋고 배달좋고 다좋은데 맛이 그닥.... 이돈이면 도미노 먹는게 나을듯.... | 2 | 1 | 5 | 5
피자알볼로-서대전점 | 가격대비 완전별로 품평보고 시

-----

In [75]:
# 파일에 데이터 덮어쓰기
f = open(r"/Users/keonkookkim/Desktop/KKK/Selenium/review.csv", 'a', encoding="UTF-8", newline="") # 파일 입출력 기능 

csvWirter = csv.writer(f) # wirter에 파일(f) 넘겨주기

In [76]:
bad_reveiw_crawling()

쌈밥잘하는집 | 돼지불고기는 먹을만한 정도였었고, 제가 거의 냉면은 다 먹을수있는데 냉면에서 잡내가 심하게나서 다 남겼고 배달은 예상배달시간이 1시간이었는데 20분정도 늦었어요. 이럴거면 다른데서 시킬걸 하고 자괴감드네요 | 1 | 1 | 2 | 2
쌈밥잘하는집 | 맛은 괜찮은데.. 배달용기좀 신경쓰셔야 할꺼같네요.. 뚜껑을 여는순간 깜짝놀랬네요 고추랑마늘 먹으라는건지 이건뭐 쌈장도 기름들어가서 이상한맛나고 지난번에도 이렇게 오더니 또이렇게 왔네요 신경좀써주세요 | 1 | 2 | 1 | 1
쌈밥잘하는집 | 김치찌게가 아니라 김치국임. 속지마세요. 김치국은 시원한 맛이라도 있지. 소금만 떼리 부어 음식물 쓰기# 버려야 할 판. 4명식구 아무도 안먹어서 다시 저녁 시켜야겠어요 | 1 | 1 | 1 | 1
쌈밥잘하는집 | 음....기름이... | 2 | 2 | 2 | 2
쌈밥잘하는집 | 생각보다 너무 맛이 없었어요 갈치조림은 외국산 갈치2조각 무2조각이였어요 반찬, 국물거리도 가격대비 부실했어요 | 2 | 2 | 2 | 2
쌈밥잘하는집 | 면이덜익구 국물이 싱거워요 | 1 | 1 | 3 | 3
쌈밥잘하는집 | 순살찜닭정식 닭비린내 남 후기에 오징어뽁음 오징어는별루없고 야채만엄청있고 맛두 머그냥그렇다고 하길래 혹시나 해서 다른메뉴시켜봤는게 아니네요 | 1 | 1 | 5 | 5
쌈밥잘하는집 | 댓글은 다 맛있다던데 저는 고기 누린내가 심해 다버렸어요ㅜㅜ | 1 | 1 | 1 | 1
쌈밥잘하는집 | disaster 🥵 | 1 | 1 | 5 | 5
쌈밥잘하는집 | 조리부터 배달까지 60분 소요된다고 해서 배달 완료 시간 맞춰 나갔는데 그 후로 30분이 더 지나서 도착했네요.. 기사님은 다른 곳에서 도착했다고 전화주시고,, 가게 잘못은 아니지만 조리 후 30분이 지나서 면은 다 불고 싱거워져서 못 먹을 맛이었습니다ㅜ | 1 | 1 | 1 | 1
쌈밥잘하는집 | 기사님은 친절했는데 음식받자마자 대부분 찌개들이 밀봉이 잘 안되었는지 넘쳐흘렸네요...근데 밀봉했늠데 찌개에 들어간

In [77]:
f.close()

In [78]:
data = pd.read_csv('/Users/keonkookkim/Desktop/KKK/Selenium/review.csv')
data.columns =['store','review','total_scope', 'taste_star','quantity_star','delivery_star']
data.shape

(5279, 6)